In [67]:
import csv
import os
import logging
import boto3
from botocore.exceptions import ClientError
import sys
import threading
from tqdm import tqdm

In [68]:
with open('credentials.csv','r') as input:
    next(input)
    reader = csv.reader(input)
    for line in reader:
        access_key_id = line[2]
        secret_access_key = line[3]

In [69]:
class ProgressPercentage(object):

    def __init__(self, filename):
        self._filename = filename
        self._size = float(os.path.getsize(filename))
        self._seen_so_far = 0
        self._lock = threading.Lock()

    def __call__(self, bytes_amount):
        # To simplify, assume this is hooked up to a single filename
        with self._lock:
            self._seen_so_far += bytes_amount
            percentage = (self._seen_so_far / self._size) * 100
            sys.stdout.write(
                "\r%s  %s / %s  (%.2f%%)" % (
                    self._filename, self._seen_so_far, self._size,
                    percentage))
            sys.stdout.flush()

In [84]:
def upload_file(path_name, file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = path + file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(path + file_name, bucket, object_name )
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [85]:
s3 = boto3.resource('s3',
                    #region_name='us-east-1',
                    region_name='sa-east-1',
                    aws_access_key_id = access_key_id,
                    aws_secret_access_key = secret_access_key)

In [86]:
#my_bucket = s3.Bucket('project-identify')

In [87]:
#count = 0
#for file in my_bucket.objects.all():
 #   count += 1 #print(file.key)
#print(count)

In [88]:
path = "upload/"
valid_images = [".jpg",".png"]
photos = os.listdir(path)

In [89]:
for photo in tqdm(photos):
    b = upload_file(path, photo, 'project-identify')

upload/_54b2899-x3.jpg  45953 / 45953.0  (100.00%)))

In [91]:
# read files uploaded
my_bucket = s3.Bucket('project-identify')
count = 0
for file in my_bucket.objects.all():
   count += 1 #print(file.key)
print(count)

745
